# Introduction to Image Classification with Transfer Learning

## What to take away from this presentation

- Python is a good choice for machine learning
- Jupyter Notebooks for easy experimentation
- Basic understanding of a Convolutional Neural Network (CNN)
- What transfer learning is
- Basic understanding of the requirements for training a CNN
- Familiarity with using the Keras framework

## What is Transfer Learning?

Taking advantage of existing work done in the image classification field.

Specifically, building on the feature extraction layers of existing CNN architectures.

To do just that we can take a network that someone else has spent a long time training and just borrow the feature extraction bits of it, already pre-trained, typically these consist of convolution and pooling layers. We attach those borrowed layers onto some additional fully connected layers (the neural network) and train the complete model on our specific dataset.

## Tools

- Keras: A popular machine leaning framework and library for Python
 - Side note: Python is my favourite programming language
- Tensorflow: A backend for Keras, as well as machine learning framework on its own.
 - Tensorflow is a library for high performance numerical computation.

## What is a CNN?

You can generally think of a CNN being used for image classification as having two major sections, responsible for feature extraction and classification.

The layers in a neural network that perform classification are usually referred to as the 'fully connected' layers, in our transfer learning example it is the weights or these layers that we'll be updating.

Initial layers in the CNN are for extracting features from the images to make the job of the fully connected layers easier (e.g. filtering, convolutions, pooling)

An in-depth explanation of neural net architectures is beyond the scope of this talk, though if you have any burning questions I can try and help you out. 

![CNN](https://cdn-images-1.medium.com/max/1200/1*XbuW8WuRrAY5pC4t-9DZAQ.jpeg)
https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148

When training a network what you are actually doing is minimising the loss function, this has to do with derivatives and simultaneous equations where you continuously make small changes to the network weights to improve the classification accuracy as you train.

# Getting into some code
We're just going to make all our python imports up front to make code in later cells cleaner

In [ ]:
# Keras Framework
import keras as K
# Numpy - matrix math library for Python written in C
import numpy as np
# Tensorflow Framework
import tensorflow as tf
# Matlab-like plotting library and
# enable intereactive jupyter notebook plote of appropriate size
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 9.5, 6
import matplotlib.pyplot as plt
# Utilities for tweaking this Jupyter Notebook
import IPython

## Obtain Dataset - CIFAR10

We'll be using the CIFAR10 dataset for the example, CIFAR10 is a dataset containing 50,000 32 by 32 RGB training images and 10,000 test images split into 10 categories.

Here's an example of some of those:

![cifar10](http://corochann.com/wp-content/uploads/2017/04/cifar10_plot_more.png)

Keras includes a copy of this dataset that's easily accessible entirely from code, so we can just import the data and begin using it (after waiting for it to download the first time we import it).

In [ ]:
(input_train_orig, truth_train_orig), (input_test_orig, truth_test_orig) = K.datasets.cifar10.load_data()

CIFAR10 provides image truth as single number in the range 0-9, but we need a categorical representation, Keras provides us a function to generate this:

In [ ]:
TRUTH_CLASSES = 10
truth_train = K.utils.to_categorical(truth_train_orig, TRUTH_CLASSES)
truth_test = K.utils.to_categorical(truth_test_orig, TRUTH_CLASSES)
print("Original Truth Format Example:   ", truth_train_orig[0])
print("Categorical Truth Format Example:", truth_train[0])

Having loaded the dataset, we can plot a preview of some of the images to check what they look like, and what they're classified as.

In [ ]:
def plot_images(images, truths, rng, preds=None):
    """Plot some images labelled with their truth, and optionally the prediction the network made"""
    
    labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    cols = 5
    rows = len(rng) // cols
    plt.figure()
    
    truths = np.argmax(truths, axis=1)
    if preds is not None:
        preds = np.argmax(preds, axis=1)

    for i, idx in enumerate(rng):
        truth = labels[truths[idx]]
        plt.subplot(rows, cols, i+1)
        title = '{truth}'.format(truth=truth)
        color = 'b'
        if preds is not None:
            color = 'g'
            pred = labels[preds[idx]]
            join = '=='
            if pred != truth:
                color = 'r'
                join = '!='
            title = '"{pred}" {join} '.format(pred=pred, join=join) + title
            
        plt.title(title, color=color)
        plt.imshow(images[idx])
    plt.tight_layout()
        
plot_images(input_train_orig, truth_train, range(15))

# Import a pre-trained Model (VGG19)

To kick start our classification we'll base our model off the VGG19 CNN model, like getting a copy of CIFAR 10, Keras provides us an easy way to download this pre-trained model.

VGG has been pre-trained on another well-known dataset, ImageNet, so the convolutional layers will already be pretty good at extracting features from the images. 

Note the `include_top` argument when importing the model is false - this is a convenience feature for transfer learning when importing models in Keras, it means that the importd model doe not include fully-connected layers, so that we can define our own and attach them easily.

In [ ]:
# The minimum image size that our pre-trained imported VGG19 supports is 48x48
# We will have to upscale our training images before we can train the network
INPUT_DIM = 48;

vgg19 = K.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(INPUT_DIM, INPUT_DIM, 3),
)

# The imported layers are already trained in ImageNet,
# So we can mark them as not trainable to prevent weights modification
for layer in vgg19.layers:
    layer.trainable = False

Keras provides a method for us to summarise a model as text, we can inspect what we've downloaded:

In [ ]:
vgg19.summary()

# Image Preprocessing

If we scroll up and look at the input layer of our model - note the expected input size of 48x48x3 (the 3 is for the RGB channels), but remember the CIFAR10 images are 32x32:

In [ ]:
print('Train Input Shape, 50000 items):', input_train_orig.shape)
print('Test Input Shape, 10000 items): ', input_test_orig.shape)

What to do? Well we can just resize all the images in our input dataset, tensorflow provides a handy helper function for just this

In [ ]:
%%time

input_train_big_tensor = tf.image.resize_images(input_train_orig, [INPUT_DIM, INPUT_DIM])
input_test_big_tensor  = tf.image.resize_images(input_test_orig, [INPUT_DIM, INPUT_DIM])

sess = tf.Session()
with sess.as_default():
    input_train_big = input_train_big_tensor.eval().astype('uint8')
    input_test_big  = input_test_big_tensor.eval().astype('uint8')
sess.close()

Now we run the resized images through a preprocessing function provided by VGG19 - this just converts the channels to BGR instead of RGB, then subtracts the average BGR values for the `ImageNet` dataset from each pixel in our input images.

In [ ]:
%%time
input_train_preprocessed = K.applications.vgg19.preprocess_input(input_train_big)
input_test_preprocessed = K.applications.vgg19.preprocess_input(input_test_big)

## Visualise the Preprocessing

Let's check that the resized images look somewhat similar to the original images:

Take a note though, that the preprocessing function has made some of the RGB channels negative (and made the data type `float32`), here I've cast the data type to `uint8` to make visualisation effective, but it means that some channels overflow.

In [ ]:
start = 5
num = 6

plt.figure()

plt.subplot(311)
plt.title('Original')
plt.imshow(np.concatenate(input_train_orig[start:start+num], axis=1))

plt.subplot(312)
plt.title('Resized')
plt.imshow(np.concatenate(input_train_big[start:start+num], axis=1))

plt.subplot(313)
plt.title('Preprocessed')
plt.imshow(np.concatenate(input_train_preprocessed[start:start+num].astype('uint8'), axis=1))

plt.tight_layout()

Just to check we understand what happened, we can look at the transformation of the top left pixel of each image.

In [ ]:
print('Original')
print(' Type:', input_train_orig.dtype)
print(' Values:', input_train_orig[0,0,0])
print('\nResized')
print(' Type:', input_train_big.dtype)
print(' Values:', input_train_big[0,0,0])
print('\nPreprocessed (Convert tp BGR and subtract Imagenet Means)')
print(' Type:', input_train_preprocessed.dtype)
print(' Values:', input_train_preprocessed[0,0,0])


IMAGENET_BGR_MEANS = [103.939, 116.779, 123.68]
print('\nManually Processed (Flipped and ImageNet means subtracted)')
print(' Values:', np.flip(input_train_big[0,0,0], axis=0) - IMAGENET_BGR_MEANS)

# Attach and Train our own Layers

Now we create our own fresh layers, attach them to the VGG19 model we imported, and train the resulting franken-model, which we can then run our testing dataset on and see how it performs!

We're first going to flatten the output of the VGG model so we have a single dimensional tensor of the last layer in the VGG model, then attach that to three fully connected layers with 200, 200, and 50 neurons respectively. Finally we will have a fully connected layer with 10 neurons (one for each class) as the output layer.

When the network produces an output for a single input, it will have 10 values between 0 and 1, and the index of the highest output is the prediction of the network.

In [ ]:
# Create a sequence of fully-connected layers that build on the imported vgg19 network
# To do this we use the keras functional API
x = K.layers.Flatten()(vgg19.output)
x = K.layers.Dense(200, activation='relu')(x)
x = K.layers.Dense(200, activation='relu')(x)
x = K.layers.Dense(50, activation='relu')(x)
output = K.layers.Dense(TRUTH_CLASSES, activation='softmax')(x)

# Wrap the layer sequence in a Keras Model, required for running the training
model = K.models.Model(vgg19.input, output)

model.summary()

Now we've put together the model we need to decide some of the parameters that will be used for training, and compile the model, which configures the tensorflow models for training.

When we compile this model we give it:
- `optimiser`: The optimiser controls the learning rate of the model, updates to the weights.
- `loss`: The type of function used to generate an measure of how well the model is performing on the data, the closet the loss measurement gets to 0, the better the model performed on a given dataset
- `metrics`: Functions used to judge the model performance, while this is similar to the loss function, they serve  different purposes and it's easier for a human to understand accuracy

In [ ]:
optimizer = K.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

The following is some boilerplate plotting code that gives us the pretty graph as we train the network

In [ ]:
class PlotLearning(K.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.epoch = 0
        self.epochs = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig, (self.ax1, self.ax2) = plt.subplots(1, 2)
        self.fig.canvas.draw()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.epochs.append(self.epoch)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc') * 100)
        self.val_acc.append(logs.get('val_acc') * 100)
        self.epoch += 1

        self.ax1.clear()
        self.ax1.set_yscale('log')
        self.ax1.plot(self.epochs, self.losses, label="Training Loss")
        self.ax1.plot(self.epochs, self.val_losses, label="Validation Loss")
        self.ax1.set_xlabel('Epochs')
        self.ax1.set_ylabel('Loss')
        self.ax1.legend()
        
        self.ax2.clear()
        self.ax2.plot(self.epochs, self.acc, label="Training, Accuracy")
        self.ax2.plot(self.epochs, self.val_acc, label="Validation Accuracy")
        self.ax2.set_xlabel('Epochs')
        self.ax2.set_ylabel('Accuracy %')
        self.ax2.legend()
        
        plt.tight_layout()
        self.fig.canvas.draw()

        
plot_callback = PlotLearning()

Finally we get to actually train the network (Keras calls this 'fitting'). 

The key things here are:
- `epochs`: One full presentation of all entries the training dataset to the network
- `batch`: How many entries to present to the network before performing a gradient update (changing weights, which is how the network really learns)
- `validation_split`: Portion of the training data set aside and not used for training, but instead used for validation (checking if we've overtrained the network)

In [ ]:
EPOCHS = 30
BATCH = 32

train_hist = model.fit(
    input_train_preprocessed,
    truth_train,
    epochs=EPOCHS,
    batch_size=BATCH,
    shuffle=True,
    verbose=1,
    validation_split=0.1,
    callbacks=[plot_callback]
)

## Evaluate Performance

It's important to check the accuracy of the model on data that wasn't part of the training process, to validate that the model has not been overtrained on the training data. This is what we use the testing set of CIFAR 10 for.

In [ ]:
test_loss, test_acc = model.evaluate(input_test_preprocessed, truth_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

# Make Some Predictions!

Creating predictions on the testing dataset and storing them:

In [ ]:
predictions = model.predict(input_test_preprocessed, verbose=1, batch_size=256)

Plot a sample of those predictions using the function we created for that purpose earlier:

In [ ]:
plot_images(input_test_orig, truth_test, range(15), preds=predictions)